# Simulation Project

**Author:** Carlos Alfredo Hernández Alvarez

**ORCID:** [https://orcid.org/0009-0006-6749-1686](https://orcid.org/0009-0006-6749-1686)

The objective of the simulation is to analyze and improve the operation of a souvenir business by evaluating waiting times, system efficiency and management strategies, using simulated data to optimize performance and customer experience.

# Problem

A souvenir store **groups of tourists arrive every 5 +/- 1 minute**. The groups consist of a guide and **15 to 20 tourists**.
The guide leaves to get tickets for the next tour, **stopping for 20 +/- 5 minutes**. Meanwhile, **tourists tour the store for 15 +/- 5 minutes**. After browsing the store, 65% of tourists go to a cashier **where they take 20 +/- 5 seconds to pay** for the items purchased.
Both those who buy and those who do not, take **10 +/- 2 seconds to leave** the store. Once the entire group (tourists and guide) is assembled, they leave for the next tour, ending the system under study. Simulate groups terminating the system:

a) Calculate the average Queue Time of the tourists shopping.

b) Calculate the average time each group spent in the system.

# Variables

## Entrance

- Arrival time of a group of tourists(`U1`) = U(4, 6) minutes

- Tourist travel time in the store(`U2`) = U(10, 20) minutes

- Time taken to pay for each tourist(`U3`) = U(15, 25) seconds

- Time of guide's absence(`U4`) = U(15, 25) minutes

- Tourist exit time(`U5`) = U(8, 12) seconds

- Group size(`U6`) = U(15, 20) tourists

## State

 - Arrival of the group of tourists(`arrival`):

 - arrival<sub>i</sub> = U<sub>1</sub> + arrival<sub>i-1</sub>

- End of the tour of the tursitas in the tent(`finish course`):

 - finish course<sub>i</sub> = U<sub>2</sub> + arrival<sub>i</sub>

- Finish paying the tour group(`finish paying`):

 - finish paying<sub>i</sub> = max(finish course<sub>i</sub>, finish paying<sub>i - 1</sub>) + 0.65 * group size<sub>i</sub> * U<sub>3</sub> / 60

- Return of the guide(`guide return`):

 - 	guide return<sub>i</sub> = U<sub>4</sub> + arrival<sub>i</sub>

- Departure of the group from the store(`out of business`):

 - out of business<sub>i</sub> = max(U<sub>5</sub> / 60 + finish paying<sub>i</sub>, guide return<sub>i</sub>)






## Output

1.Average time in queue(`queue mean`):
 - Time in queue(`queue time`)

- If finish course<sub>i</sub> > finish paying<sub>i - 1</sub>:
    - queue time<sub>i</sub> = finish paying<sub>i - 1</sub> - finish course<sub>i</sub>
- queue mean = Σ(queue time)/n
<br>
<br>

2.Average time in system (`time in system mean`):
- Time in queue(`system time`)

- system time<sub>i</sub> = out of business<sub>i</sub> - arrival<sub>i</sub>
- time in system mean
 = Σ(system time)/n

# Simulation

## Raw simulation

Implementation of the raw simulation, a function is defined for each state variable.

### Initialize

In [1]:
import random
import pandas as pd
import numpy as np

In [2]:
arrival_time_tourists = 5
desviation_arrival_time_tourists = 1

time_absence_guide = 20
desviation_time_absence_guide = 5

time_traveled = 15
desviation_time_traveled = 5

payment_time = 20
desviation_payment_time = 5

output_time = 10
desviation_output_time = 2

group_size_min = 15
group_size_max = 20

data = pd.DataFrame(columns=['group size', 'U1', 'arrival', 'U2', 'finish course',
                             'U3', 'finish paying', 'U4', 'guide return', 'U5', 'out of business', 'system time'])

In [3]:
data

,group size,U1,arrival,U2,finish course,U3,finish paying,U4,guide return,U5,out of business,system time


In [4]:
def arrival(id, reductuion=False, n=0):
  max = arrival_time_tourists + desviation_arrival_time_tourists
  min = arrival_time_tourists - desviation_arrival_time_tourists

  if reductuion:
    data.loc[id, 'U1'] = max - data.loc[id - n, 'U1'] + min
  else:
    data.loc[id, 'U1'] = random.uniform(min, max)

  if id == 0:
    data.loc[id, 'arrival'] = data.loc[id, 'U1']
  else:
    data.loc[id, 'arrival'] = data.loc[(id - 1), 'arrival'] + data.loc[id, 'U1']

In [5]:
def finish_course(id, reductuion=False, n=0):
  max = time_traveled + desviation_time_traveled
  min = time_traveled - desviation_time_traveled

  if reductuion:
    data.loc[id, 'U2'] = max - data.loc[id - n, 'U2'] + min
  else:
    data.loc[id, 'U2'] = random.uniform(min, max)

  data.loc[id, 'finish course'] = data.loc[id, 'U2'] + data.loc[id, 'arrival']

In [6]:
def finish_paying(id, reductuion=False, n=0):
  max = payment_time + desviation_payment_time
  min = payment_time - desviation_payment_time

  if reductuion:
    data.loc[id, 'U3'] = max - data.loc[id - n, 'U3'] + min
  else:
    data.loc[id, 'U3'] = random.uniform(min, max)

    U3 = list()

    for i in range(data.loc[id, 'group size']):
      U3.append(random.uniform(min, max))

    data.loc[id, 'U3'] = np.mean(U3)

  if id == 0:
    data.loc[id, 'finish paying'] = (data.loc[id, 'finish course'] + 0.65 *
                                     data.loc[id, 'group size'] * data.loc[id, 'U3'] / 60)
  else:
    data.loc[id, 'finish paying'] = (np.max([data.loc[id, 'finish course'], data.loc[(id - 1), 'finish paying']]) +
                                     0.65 * data.loc[id, 'group size'] * data.loc[id, 'U3'] / 60)

In [7]:
def guide_return(id, reductuion=False, n=0):
  max = time_absence_guide + desviation_time_absence_guide
  min = time_absence_guide - desviation_time_absence_guide

  if reductuion:
    data.loc[id, 'U4'] = max - data.loc[id - n, 'U4'] + min
  else:
    data.loc[id, 'U4'] = random.uniform(min, max)

  data.loc[id, 'guide return'] = data.loc[id, 'U4'] + data.loc[id, 'arrival']

In [8]:
def out_of_business(id, reductuion=False, n=0):
  max = output_time + desviation_output_time
  min = output_time - desviation_output_time

  if reductuion:
    data.loc[id, 'U5'] = max - data.loc[id - n, 'U5'] + min
  else:

    U5 = list()

    for i in range(data.loc[id, 'group size']):
      U5.append(random.uniform(min, max))

    data.loc[id, 'U5'] = np.mean(U5)

  if id == 0:
    data.loc[id, 'out of business'] = np.max([data.loc[id, 'guide return'],
                                            data.loc[id, 'group size'] * data.loc[id, 'U5'] / 60])
  else:
    data.loc[id, 'out of business'] = np.max([data.loc[id, 'guide return'], data.loc[id, 'finish paying'] +
                                              data.loc[id, 'group size'] * data.loc[id, 'U5'] / 60])

In [9]:
def begin(id, reduction=False, n=0):
  arrival(id, reduction, n)
  finish_course(id, reduction, n)
  finish_paying(id, reduction, n)
  guide_return(id, reduction, n)
  out_of_business(id, reduction, n)

In [10]:
def main(register=100, reduction=False):
  if reduction:
    register = int(register/2)

  for i in range(register):
    data.loc[i, 'group size'] = int(random.randint(group_size_min, group_size_max))
    begin(i)

  if reduction:
    for i in range(register):
      data.loc[i + register, 'group size'] = group_size_max - data.loc[i, 'group size'] + group_size_min
      begin(i + register, True, register)

  data['system time'] = data['out of business'] - data['arrival']

  return data.copy()

In [11]:
def to_time(col):
    c = col.copy()

    for i in range(len(c)):
        hour = int(c[i] // 60)
        sec = c[i] % 1
        min = int((c[i] - sec) % 60)
        sec = round(sec * 60)
        c[i] = f'{hour:02d}:{min:02d}:{sec:02d}'

    return c.copy()

In [12]:
def to_see(df):
  columns = df.columns[1:]

  for col in columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

  df = df.round(2)

  df['arrival'] = to_time(df['arrival'])
  df['finish course'] = to_time(df['finish course'])
  df['finish paying'] = to_time(df['finish paying'])
  df['guide return'] = to_time(df['guide return'])
  df['out of business'] = to_time(df['out of business'])
  df['system time'] = to_time(df['system time'])

  return df.copy()

In [13]:
main(5)

,group size,U1,arrival,U2,finish course,U3,finish paying,U4,guide return,U5,out of business,system time
0,15,4.31964,4.31964,15.996258,20.315898,20.978448,23.724896,21.834715,26.154355,9.869479,26.154355,21.834715
1,17,5.866303,10.185943,16.837068,27.023011,19.638918,30.639845,23.688695,33.874639,10.406916,33.874639,23.688695
2,18,4.868577,15.054521,18.408886,33.463406,19.75636,37.315897,22.294578,37.349099,10.056839,40.332948,25.278428
3,15,5.005288,20.059809,11.439778,31.499587,19.430516,40.473356,23.010418,43.070227,9.735552,43.070227,23.010418
4,20,4.726713,24.786522,17.022619,41.809141,19.617703,46.059643,23.580983,48.367505,9.404016,49.194315,24.407794


In [14]:
example = main(register=100)

def output():
  qt = list()
  for i in range(1, example.shape[0]):
    if example.loc[i-1, 'finish paying'] == np.max([example.loc[i, 'finish course'], example.loc[i-1, 'finish paying']]):
      qt.append(example.loc[i-1, 'finish paying'] - example.loc[i, 'finish course'])

  output1 = f"Average queuing time: {np.round(np.mean(qt),2)} minutes."
  output2 = f"Average time in the system: {np.round(np.mean(example['system time']),2)} minutos."
  return (output1, output2)

### Example of a simulation

In [15]:
to_see(example)

,group size,U1,arrival,U2,finish course,U3,finish paying,U4,guide return,U5,out of business,system time
0,20,4.25,00:04:15,10.65,00:14:55,19.81,00:19:12,15.75,00:20:01,9.65,00:20:01,00:15:45
1,19,4.04,00:08:17,15.09,00:23:23,19.20,00:27:20,22.95,00:31:14,10.22,00:31:14,00:22:57
2,16,4.92,00:13:13,10.97,00:24:11,19.72,00:30:45,17.35,00:30:34,10.52,00:33:33,00:20:20
3,17,5.32,00:18:32,15.66,00:34:11,19.17,00:37:43,21.23,00:39:46,10.72,00:40:46,00:22:14
4,18,4.77,00:23:18,15.76,00:39:04,20.01,00:42:58,24.93,00:48:14,9.98,00:48:14,00:24:56
...,...,...,...,...,...,...,...,...,...,...,...,...
95,15,4.71,07:53:20,12.25,08:05:35,19.95,08:11:58,17.12,08:10:28,9.93,08:14:26,00:21:06
96,16,5.72,07:59:03,17.32,08:16:22,18.33,08:19:33,22.84,08:21:54,9.93,08:22:11,00:23:08
97,18,5.59,08:04:38,19.82,08:24:28,19.83,08:28:20,24.41,08:29:03,10.01,08:31:20,00:26:41
98,19,5.27,08:09:55,18.93,08:28:51,20.58,08:33:05,24.94,08:34:52,9.94,08:36:14,00:26:19


### Output

In [16]:
output1, output2 = output()
print(output1)
print(output2)

Average queuing time: 2.98 minutes.
Average time in the system: 24.16 minutos.


## Statistical measures

### Initialize

In [17]:
def iterations(n, r=100, reduction=False):
  simulations = list()

  for i in range(n):
    simulations.append(main(register=r, reduction=reduction))

  return simulations

In [18]:
def statistics(simulations):
  from scipy import stats

  _ = pd.DataFrame(columns=['queue mean', 'standard error(QM)', 'interval(QM)', 'time in system mean', 'standard error(TS)', 'interval(TS)'])

  for i, sim in enumerate(simulations):
    queue_time = list()

    for j in range(1, sim.shape[0]):
      if sim.loc[j-1, 'finish paying'] == np.max([sim.loc[j, 'finish course'], sim.loc[j-1, 'finish paying']]):
        queue_time.append(sim.loc[j-1, 'finish paying'] - sim.loc[j, 'finish course'])

    _.loc[i, 'queue mean'] = np.mean(queue_time)
    _.loc[i, 'standard error(QM)'] = np.mean(stats.sem(_['queue mean']))
    _.loc[i, 'interval(QM)'] = stats.t.interval(0.95, len(sim)-1, loc=_.loc[i, 'queue mean'],
                                                 scale=_.loc[i, 'standard error(QM)'])

    _.loc[i, 'time in system mean'] = np.mean(sim['system time'])
    _.loc[i, 'standard error(TS)'] = np.mean(stats.sem(sim['system time']))
    _.loc[i, 'interval(TS)'] = stats.t.interval(0.95, len(sim)-1, loc=_.loc[i, 'time in system mean'],
                                                 scale=_.loc[i, 'standard error(TS)'])

  return _.copy()

In [19]:
def info(col, interval):
  _ = {
      'target' : [col.name],
       'mean' : [col.mean()],
       'std' :[col.std()],
       'variance' : [col.var()],
       'interval' : [[interval.apply(lambda x: x[0]).mean(),
                      interval.apply(lambda x: x[1]).mean()]]
       }

  return pd.DataFrame(_)

In [20]:
def stats_mean(target):
  df_stats = statistics(target)

  df_stats_mean = pd.concat([info(df_stats['queue mean'], df_stats['interval(QM)']),
                            info(df_stats['time in system mean'], df_stats['interval(TS)'])],
                            ignore_index=True)

  df_stats_mean['interval'] = df_stats_mean['interval'].apply(lambda x: np.round(x, 3))
  df_stats_mean = np.round(df_stats_mean, 3)

  return df_stats_mean.copy()

In [21]:
def test(i=100, n=100):
  df_iterations = iterations(i,r=n)
  df_iterations_stats = stats_mean(df_iterations)

  df_iterations_var = iterations(i, n, True)
  df_iterations_var_stats = stats_mean(df_iterations_var)

  return (df_iterations, df_iterations_stats, df_iterations_var, df_iterations_var_stats)

In [22]:
import warnings

warnings.filterwarnings('ignore')
x = test(100, 100)

### Output

Statistical measures for 100 simulations with 100 groups of tourists per simulation.

In [23]:
x[1]

,target,mean,std,variance,interval
0,queue mean,3.158,0.334,0.111,"[3.049, 3.273]"
1,time in system mean,23.735,0.250,0.063,"[23.378, 24.093]"


## Complementary numbers

### Example

Applying variance reduction for 100 simulations with 100 groups of tourists per simulation.

In [24]:
to_see(x[2][1])

,group size,U1,arrival,U2,finish course,U3,finish paying,U4,guide return,U5,out of business,system time
0,18,5.35,00:05:21,17.97,00:23:19,20.09,00:27:14,22.69,00:28:02,9.99,00:28:02,00:22:41
1,16,4.44,00:09:47,12.93,00:22:43,19.39,00:30:36,23.11,00:32:54,10.16,00:33:19,00:23:31
2,16,4.97,00:14:46,13.43,00:28:11,19.88,00:34:02,21.24,00:35:59,10.25,00:36:47,00:22:01
3,15,4.62,00:19:23,10.56,00:29:56,20.30,00:37:20,16.46,00:35:50,10.39,00:39:56,00:20:34
4,19,5.39,00:24:46,16.50,00:41:16,20.24,00:45:26,17.02,00:41:47,10.25,00:48:41,00:23:55
...,...,...,...,...,...,...,...,...,...,...,...,...
95,15,4.63,08:00:43,13.51,08:14:13,18.43,08:18:46,16.19,08:16:55,10.06,08:21:17,00:20:34
96,17,4.63,08:05:20,19.94,08:25:17,20.68,08:29:05,22.46,08:27:49,10.23,08:31:59,00:26:38
97,17,4.69,08:10:02,16.53,08:26:34,18.72,08:32:32,18.02,08:28:04,10.03,08:35:23,00:25:20
98,16,4.11,08:14:08,15.81,08:29:57,20.52,08:36:05,22.52,08:36:40,9.66,08:38:40,00:24:32


### Statistical measures

In [25]:
x[3]

,target,mean,std,variance,interval
0,queue mean,3.145,0.373,0.139,"[3.017, 3.264]"
1,time in system mean,23.716,0.200,0.040,"[23.354, 24.078]"


In [26]:
'\U0001F600'

'😀'

# Conclusions

This souvenir business simulation project has provided key findings on the effective operation and management of the business. Through the simulation, we have been able to identify critical areas for improvement, such as waiting times and system efficiency during tourist visits. This is crucial to optimizing the customer experience and ensuring that the business runs smoothly and efficiently.

One of the most significant findings has been the importance of variance reduction techniques, especially complementary numbers. These techniques are critical because they help reduce the uncertainty in the estimates obtained during simulation. By minimizing variance, more accurate and reliable results are obtained, which is crucial for making informed decisions about queue management, resource allocation, and staff schedules.

In addition, simulation has validated the effectiveness of different operational strategies before implementing them in the real business environment. This provides a significant advantage by allowing us to test different scenarios and policies without associated risks and costs. Thus, we can identify best practices to improve operational efficiency and ensure a positive customer experience.